# Predicting Diabetes Patient Readmission

Data derived from the Health Facts data we used was an extract representing
10 years (1999–2008) of clinical care at 130 hospitals andintegrated delivery networks throughout the United States:
Midwest (18 hospitals), Northeast (58), South (28), and West (16). 
(Impact of HbA1c Measurement on Hospital Readmission Rates: Analysis of 70,000 Clinical Database Patient Records)

__Imports__

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../dataset_diabetes/diabetic_data.csv', na_values='?')
df.head()

/Users/aishakamara/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


[attribute details](https://www.hindawi.com/journals/bmri/2014/781670/tab1/)

In [3]:
df.shape

(101766, 50)

In [4]:
df.dtypes

encounter_id                 int64
patient_nbr                  int64
race                        object
gender                      object
age                         object
weight                      object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
payer_code                  object
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride         

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      99493 non-null   object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    3197 non-null    object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                61510 non-null   object
 11  medical_specialty         51817 non-null   object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

__Missing Observations__

In [6]:
df.isnull().sum()

encounter_id                    0
patient_nbr                     0
race                         2273
gender                          0
age                             0
weight                      98569
admission_type_id               0
discharge_disposition_id        0
admission_source_id             0
time_in_hospital                0
payer_code                  40256
medical_specialty           49949
num_lab_procedures              0
num_procedures                  0
num_medications                 0
number_outpatient               0
number_emergency                0
number_inpatient                0
diag_1                         21
diag_2                        358
diag_3                       1423
number_diagnoses                0
max_glu_serum                   0
A1Cresult                       0
metformin                       0
repaglinide                     0
nateglinide                     0
chlorpropamide                  0
glimepiride                     0
acetohexamide 

Per our results above, let's take a closer look at all variables that have missing/null values. 

__Weight__

In [7]:
df.weight.isnull().sum()

98569

In [8]:
df.weight.isnull().mean()

0.9685847925633315

Approximately 97% or more than 9,500 of the values denoted under the weight variable are missing or null values. Typically in cases of missing values the next step is to determine which method to best impute the missing values but seeing as over 95% of the values are missing for a data set with over 100,000 observations, it makes much more sense to simply drop the column all together.

In [9]:
df.drop('weight', axis =1 , inplace = True)

__Payer Code__

- Find article explaining why to drop.

In [10]:
df.payer_code.unique()

array([nan, 'MC', 'MD', 'HM', 'UN', 'BC', 'SP', 'CP', 'SI', 'DM', 'CM',
       'CH', 'PO', 'WC', 'OT', 'OG', 'MP', 'FR'], dtype=object)

In [11]:
df.payer_code.value_counts(normalize=True)

MC    0.527378
HM    0.102000
SP    0.081401
BC    0.075679
MD    0.057422
CP    0.041180
UN    0.039798
CM    0.031491
OG    0.016794
PO    0.009624
DM    0.008925
CH    0.002374
WC    0.002195
OT    0.001544
MP    0.001284
SI    0.000894
FR    0.000016
Name: payer_code, dtype: float64

In [12]:
df.payer_code.isnull().sum()

40256

In [13]:
df.payer_code.isnull().mean()

0.395574160328597

With almost 40% of its data missing, I also decided to drop this variable. 

In [14]:
df.drop('payer_code',axis=1, inplace = True )

In [15]:
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')

__Medical_Specialty__

In [16]:
df.medical_specialty.unique()

array(['Pediatrics-Endocrinology', nan, 'InternalMedicine',
       'Family/GeneralPractice', 'Cardiology', 'Surgery-General',
       'Orthopedics', 'Gastroenterology',
       'Surgery-Cardiovascular/Thoracic', 'Nephrology',
       'Orthopedics-Reconstructive', 'Psychiatry', 'Emergency/Trauma',
       'Pulmonology', 'Surgery-Neuro',
       'Obsterics&Gynecology-GynecologicOnco', 'ObstetricsandGynecology',
       'Pediatrics', 'Hematology/Oncology', 'Otolaryngology',
       'Surgery-Colon&Rectal', 'Pediatrics-CriticalCare', 'Endocrinology',
       'Urology', 'Psychiatry-Child/Adolescent', 'Pediatrics-Pulmonology',
       'Neurology', 'Anesthesiology-Pediatric', 'Radiology',
       'Pediatrics-Hematology-Oncology', 'Psychology', 'Podiatry',
       'Gynecology', 'Oncology', 'Pediatrics-Neurology',
       'Surgery-Plastic', 'Surgery-Thoracic',
       'Surgery-PlasticwithinHeadandNeck', 'Ophthalmology',
       'Surgery-Pediatric', 'Pediatrics-EmergencyMedicine',
       'PhysicalMedicineandRe

In [17]:
df.medical_specialty.isnull().sum()

49949

In [18]:
df.medical_specialty.isnull().mean()

0.49082208203132677

* For the missing null values I researched to see if they indicated a different type of specialist to those listed above or it simply meant that there was no data available for that instance. Found out it was the former. Decided to drop this variable seeing that almost 50% of the data was not available to begin with. Also trying to impute these missing values would introduce more inaccuracy since I'd simply have to make speculations for the missing data.  

In [19]:
df.drop('medical_specialty',axis=1, inplace = True )

In [20]:
# from sklearn.preprocessing import LabelEncoder

In [21]:
# encode = LabelEncoder()

In [22]:
# encode.fit(df['medical_specialty'])

In [23]:
# df['medical_specialty_no'] = encode.transform(df['medical_specialty'])

[code source](https://stackoverflow.com/questions/40142686/converting-non-numeric-to-numeric-value-using-panda-libraries)

__Race__

In [24]:
df.race.unique()

array(['Caucasian', 'AfricanAmerican', nan, 'Other', 'Asian', 'Hispanic'],
      dtype=object)

In [25]:
df.race.isnull().sum()

2273

In [26]:
(df.race.isnull().mean())*100

2.2335554114340743

Approximately 2% or less than 2,300 of the values denoted under race are missing or null values. As dropping these won't result in a great impact to the overall dataset, that's the step I'll take. 

In [27]:
df.dropna(subset=['race'], inplace = True)

__Diagnoses__

In [28]:
df.diag_1.isnull().sum()

19

In [29]:
df.diag_1.isnull().mean()

0.00019096820881871088

In [30]:
df.diag_2.isnull().sum()

336

In [31]:
df.diag_2.isnull().mean()

0.0033771220085835186

In [32]:
df.diag_3.isnull().sum()

1349

In [33]:
df.diag_3.isnull().mean()

0.013558742826128472

As each of the diagnoses variables have missing or null that is less that 2% of the remaining observations, dropping these observations won't result in a great impact to the overall dataset. 

In [34]:
df.dropna(subset=['diag_1','diag_2','diag_3'], inplace = True)

In [35]:
df.shape

(98053, 47)

In [36]:
(98053/101766)*100

96.35143368119017

Summary:

* Dropped the following columns:
    - weight
    - payer_code
    - medical_speciality

* Dropped rows with  Nan/missing values for the following columns:
    - race
    - diag_1
    - diag_2
    - diag_3

We are left with 98,053 observations or 96% of our original data set to continue cleaning. 

## Looking at non numeric values

In [37]:
df.dtypes

encounter_id                 int64
patient_nbr                  int64
race                        object
gender                      object
age                         object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride                 object
acetohexamide               object
glipizide                   object
glyburide           

For a majority, if not all of the non-numeric varaibles my aim is to change them into numeric values in order for me to perform EDA on as many of the variables in the dataset as possible. 

__Race__

In [38]:
df.race.unique()

array(['Caucasian', 'AfricanAmerican', 'Other', 'Asian', 'Hispanic'],
      dtype=object)

In [39]:
df['race']= df.race.map({'Caucasian':0, 'AfricanAmerican':1, 'Asian':2, 'Hispanic':3,'Other':4}) 

In [40]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
1,149190,55629189,0,Female,[10-20),1,1,7,3,59,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,1,Female,[20-30),1,1,7,2,11,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,0,Male,[30-40),1,1,7,2,44,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,0,Male,[40-50),1,1,7,1,51,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,35754,82637451,0,Male,[50-60),2,1,2,3,31,...,No,Steady,No,No,No,No,No,No,Yes,>30


__Gender__

In [41]:
df.gender.value_counts()

Female             52833
Male               45219
Unknown/Invalid        1
Name: gender, dtype: int64

In [42]:
df.drop(df.index[df['gender'] == 'Unknown/Invalid'], inplace = True)

In [43]:
df['gender']= df.gender.map({'Female':0, 'Male':1})

In [44]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
1,149190,55629189,0,0,[10-20),1,1,7,3,59,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,1,0,[20-30),1,1,7,2,11,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,0,1,[30-40),1,1,7,2,44,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,0,1,[40-50),1,1,7,1,51,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,35754,82637451,0,1,[50-60),2,1,2,3,31,...,No,Steady,No,No,No,No,No,No,Yes,>30


__Age__

In [45]:
df.age.value_counts()

[70-80)     25305
[60-70)     21809
[80-90)     16702
[50-60)     16697
[40-50)      9265
[30-40)      3548
[90-100)     2717
[20-30)      1478
[10-20)       466
[0-10)         65
Name: age, dtype: int64

In [46]:
#dummify
df['age_range']= df.age.map({'[0-10)':0, '[10-20)':1, '[20-30)':2, '[90-100)':3,'[30-40)':4, '[40-50)':5, '[50-60)':6,'[80-90)':7,'[60-70)':8, '[70-80)':9}) 

In [47]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,age_range
1,149190,55629189,0,0,[10-20),1,1,7,3,59,...,Up,No,No,No,No,No,Ch,Yes,>30,1
2,64410,86047875,1,0,[20-30),1,1,7,2,11,...,No,No,No,No,No,No,No,Yes,NO,2
3,500364,82442376,0,1,[30-40),1,1,7,2,44,...,Up,No,No,No,No,No,Ch,Yes,NO,4
4,16680,42519267,0,1,[40-50),1,1,7,1,51,...,Steady,No,No,No,No,No,Ch,Yes,NO,5
5,35754,82637451,0,1,[50-60),2,1,2,3,31,...,Steady,No,No,No,No,No,No,Yes,>30,6


__Diagnosis__

* 'diag_1'- primary diagnoses
* 'diag_2'- secondary diagnosis
* 'diag_3'- additional diagnosis

Each of these inidcated a particular level of diagnosis for a give patient. Hoeever each had 848-923 distinct  values coded as the first three digit of teh Internatin classification of Dieseas (ICD- 9). Based on reserach papers used for my analyze I decided to do what previous reserahcer and analyst had done and conslidate these into 9 majo categories: include Circulatory, Respiratory, Digestive, Diabetes, Injury, Musculoskeletal, Genitourinary, Neoplasms, and Others.

In [48]:
diag_columns = ['diag_1', 'diag_2', 'diag_3']

In [49]:
df.dtypes

encounter_id                 int64
patient_nbr                  int64
race                         int64
gender                       int64
age                         object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride                 object
acetohexamide               object
glipizide                   object
glyburide           

In [50]:
df.diag_3.unique()

array(['255', 'V27', '403', '250', 'V45', '38', '486', '996', '197',
       '250.6', '427', '627', '414', '416', '582', 'V43', '250.01', '263',
       '250.42', '276', '482', '401', '250.41', '585', '781', '278',
       '998', '428', '568', '682', '250.02', '305', '707', '496', '599',
       '424', '518', '553', '794', '411', 'V42', '531', '511', '490',
       '562', '250.8', '250.7', '250.52', '784', '491', '581', '420',
       '724', '730', '789', '131', '250.82', '999', '41', '493', '250.03',
       '753', '786', '529', 'E888', '425', '595', '303', '560', '711',
       '8', '492', '332', '296', '438', '362', '250.4', '654', '244',
       'V70', '737', '625', '681', '250.51', '404', 'V10', '810', '280',
       '440', '785', '569', '272', '997', '250.43', '918', '584', '54',
       '788', '426', '722', '250.92', '196', '461', '535', '787', '891',
       '284', '458', '648', '780', '182', '285', '593', '413', '664',
       '564', '201', '356', 'V15', '292', '782', '715', '473', '455',


In [51]:
df['diag_1'] = df.diag_1.map(lambda x: x.replace('E','2000').replace('V', '5000'))

In [52]:
df['diag_2'] = df.diag_2.map(lambda x: x.replace('E','2000').replace('V', '5000'))

In [53]:
df['diag_3'] = df.diag_3.map(lambda x: x.replace('E','2000').replace('V', '5000'))

In [54]:
df.diag_1.unique()

array(['276', '648', '8', '197', '414', '428', '398', '434', '250.7',
       '157', '518', '999', '410', '682', '402', '500057', '189', '786',
       '427', '996', '277', '584', '462', '473', '411', '174', '486',
       '998', '511', '432', '626', '295', '196', '250.6', '182', '845',
       '423', '808', '250.4', '722', '403', '784', '707', '440', '151',
       '715', '997', '198', '564', '38', '590', '578', '250.32', '433',
       '569', '185', '536', '255', '250.13', '599', '558', '574', '491',
       '560', '244', '250.03', '577', '730', '188', '824', '250.8', '332',
       '562', '291', '296', '510', '401', '263', '438', '70', '642',
       '571', '738', '593', '618', '250.42', '807', '456', '446', '572',
       '575', '250.41', '250.02', '820', '515', '780', '250.22', '995',
       '235', '250.82', '721', '787', '556', '162', '724', '282',
       '250.83', '514', '500055', '281', '250.33', '530', '466', '435',
       '250.12', '500053', '789', '566', '822', '191', '557', '733',
  

In [55]:
df.diag_2.unique()

array(['250.01', '250', '250.43', '157', '411', '492', '427', '198',
       '403', '288', '998', '507', '174', '425', '401', '715', '496',
       '428', '585', '250.02', '410', '999', '996', '135', '244', '41',
       '571', '276', '997', '599', '424', '491', '553', '707', '286',
       '440', '493', '242', '70', '500045', '250.03', '357', '511', '196',
       '396', '197', '414', '250.52', '577', '535', '413', '53', '780',
       '518', '566', '250.6', '867', '285', '486', '500015', '8', '788',
       '340', '574', '581', '228', '530', '250.82', '786', '294', '567',
       '785', '512', '305', '250.51', '280', '648', '560', '444', '38',
       '500010', '578', '277', '781', '250.42', '426', '584', '402',
       '153', '272', '733', '34', '881', '203', '250.41', '250.13', '293',
       '250.12', '787', '342', '573', '626', '303', '250.53', '458',
       '710', '415', '500042', '284', '569', '729', '759', '682', '112',
       '292', '435', '290', '250.93', '642', '398', '319', '711',
  

In [56]:
df.diag_3.unique()

array(['255', '500027', '403', '250', '500045', '38', '486', '996', '197',
       '250.6', '427', '627', '414', '416', '582', '500043', '250.01',
       '263', '250.42', '276', '482', '401', '250.41', '585', '781',
       '278', '998', '428', '568', '682', '250.02', '305', '707', '496',
       '599', '424', '518', '553', '794', '411', '500042', '531', '511',
       '490', '562', '250.8', '250.7', '250.52', '784', '491', '581',
       '420', '724', '730', '789', '131', '250.82', '999', '41', '493',
       '250.03', '753', '786', '529', '2000888', '425', '595', '303',
       '560', '711', '8', '492', '332', '296', '438', '362', '250.4',
       '654', '244', '500070', '737', '625', '681', '250.51', '404',
       '500010', '810', '280', '440', '785', '569', '272', '997',
       '250.43', '918', '584', '54', '788', '426', '722', '250.92', '196',
       '461', '535', '787', '891', '284', '458', '648', '780', '182',
       '285', '593', '413', '664', '564', '201', '356', '500015', '292',
    

In [57]:
df['diag_1'] = df.diag_1.astype('float')

In [58]:
df['diag_2'] = df.diag_2.astype('float')

In [59]:
df['diag_3'] = df.diag_3.astype('float')

In [60]:
df.dtypes

encounter_id                  int64
patient_nbr                   int64
race                          int64
gender                        int64
age                          object
admission_type_id             int64
discharge_disposition_id      int64
admission_source_id           int64
time_in_hospital              int64
num_lab_procedures            int64
num_procedures                int64
num_medications               int64
number_outpatient             int64
number_emergency              int64
number_inpatient              int64
diag_1                      float64
diag_2                      float64
diag_3                      float64
number_diagnoses              int64
max_glu_serum                object
A1Cresult                    object
metformin                    object
repaglinide                  object
nateglinide                  object
chlorpropamide               object
glimepiride                  object
acetohexamide                object
glipizide                   

__Glucose serum test Result__

In [61]:
df.max_glu_serum.value_counts()

None    92844
Norm     2532
>200     1449
>300     1227
Name: max_glu_serum, dtype: int64

In [62]:
df['max_glu_serum']= df.max_glu_serum.map({'None':0, 'Norm':1, '>200':2, '>300':3}) 

__A1c test result__

In [63]:
df.A1Cresult.value_counts()

None    81859
>8       7631
Norm     4854
>7       3708
Name: A1Cresult, dtype: int64

In [64]:
df['A1Cresult']= df.A1Cresult.map({'None':0, 'Norm':1, '>7':2, '>8':3}) 

__Dummifying the generic drugs variables__

In [65]:
df.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'diag_1', 'diag_2', 'diag_3', 'number_diagnoses',
       'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide',
       'chlorpropamide', 'glimepiride', 'acetohexamide', 'glipizide',
       'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
       'miglitol', 'troglitazone', 'tolazamide', 'examide', 'citoglipton',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'age_range'],
      dtype='object')

In [66]:
columns = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 
        'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide','pioglitazone', 
          'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone','tolazamide', 'examide', 'citoglipton', 'insulin',
           'glyburide-metformin', 'glipizide-metformin','glimepiride-pioglitazone', 'metformin-rosiglitazone','metformin-pioglitazone']

In [67]:
def generic_medication(df, targetcols):
    for i in targetcols:
        df[i] = df[i].map({'No':0, 'Steady':1, 'Up':2, 'Down':3})

In [68]:
generic_medication(df,columns)

In [69]:
df.metformin.value_counts()

0    78807
1    17677
2     1017
3      551
Name: metformin, dtype: int64

In [70]:
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,...,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,age_range
1,149190,55629189,0,0,[10-20),1,1,7,3,59,...,2,0,0,0,0,0,Ch,Yes,>30,1
2,64410,86047875,1,0,[20-30),1,1,7,2,11,...,0,0,0,0,0,0,No,Yes,NO,2
3,500364,82442376,0,1,[30-40),1,1,7,2,44,...,2,0,0,0,0,0,Ch,Yes,NO,4
4,16680,42519267,0,1,[40-50),1,1,7,1,51,...,1,0,0,0,0,0,Ch,Yes,NO,5
5,35754,82637451,0,1,[50-60),2,1,2,3,31,...,1,0,0,0,0,0,No,Yes,>30,6


__Change in medication__

In [71]:
df.change.unique() 

array(['Ch', 'No'], dtype=object)

In [72]:
df['change']= df.change.map({'Ch':0, 'No':1})

__Diabetes Medication Prescribed__

In [73]:
df['diabetesMed']= df.diabetesMed.map({'No':0, 'Yes':1})

__Readmission__

In [74]:
df.readmitted.unique()

array(['>30', 'NO', '<30'], dtype=object)

In [75]:
df['readmitted']= df.readmitted.map({'NO':0, '<30':1, '>30':2}) 

__Sanity Check__

In [76]:
df.isnull().sum()

encounter_id                0
patient_nbr                 0
race                        0
gender                      0
age                         0
admission_type_id           0
discharge_disposition_id    0
admission_source_id         0
time_in_hospital            0
num_lab_procedures          0
num_procedures              0
num_medications             0
number_outpatient           0
number_emergency            0
number_inpatient            0
diag_1                      0
diag_2                      0
diag_3                      0
number_diagnoses            0
max_glu_serum               0
A1Cresult                   0
metformin                   0
repaglinide                 0
nateglinide                 0
chlorpropamide              0
glimepiride                 0
acetohexamide               0
glipizide                   0
glyburide                   0
tolbutamide                 0
pioglitazone                0
rosiglitazone               0
acarbose                    0
miglitol  

In [77]:
df.dtypes

encounter_id                  int64
patient_nbr                   int64
race                          int64
gender                        int64
age                          object
admission_type_id             int64
discharge_disposition_id      int64
admission_source_id           int64
time_in_hospital              int64
num_lab_procedures            int64
num_procedures                int64
num_medications               int64
number_outpatient             int64
number_emergency              int64
number_inpatient              int64
diag_1                      float64
diag_2                      float64
diag_3                      float64
number_diagnoses              int64
max_glu_serum                 int64
A1Cresult                     int64
metformin                     int64
repaglinide                   int64
nateglinide                   int64
chlorpropamide                int64
glimepiride                   int64
acetohexamide                 int64
glipizide                   

In [78]:
df.shape

(98052, 48)

In [79]:
df.to_csv('../dataset_diabetes/diabetic_data_clean.csv')